In [9]:
import time
import json
import shutil
import os
import sys
# import tldextract
# import collections
import pandas as pd
# from tqdm import tqdm
# import urllib.request
import zlib
import json
import random
import requests
# import util as u

In [6]:
def extract_file(file_path):
    file_suffix = file_path.split('.')[-1]
    unzipped_path = file_path.replace('.' + file_suffix, '')
    
    if not os.path.exists(unzipped_path):
        with gzip.open(file_path, 'rb') as f_in:
            with open(unzipped_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    
    return unzipped_path


def read_every_line(fname, max_lines=-1):
    lines = []
    with open(fname, encoding='utf-8') as f:
        for i, line in enumerate(f):
            lines.append(line.replace('\n',''))
            if i > max_lines and max_lines > 0:
                break 
    return lines


def decompress_stream(stream):
    o = zlib.decompressobj(16 + zlib.MAX_WBITS)

    for chunk in stream:
        yield o.decompress(chunk)

    yield o.flush()

In [10]:
# Set variables for Paths File and remote / web location
yearmonth = '2019-47'
url_prefix = 'https://commoncrawl.s3.amazonaws.com/'
paths_file_name = 'cc-index.paths.gz'

# Variables derives from the above
# storage_folder = './crawl-data/CC-MAIN-' + yearmonth
storage_folder = '../data/raw/'
paths_file = os.path.join(storage_folder, paths_file_name)
paths_file_unzipped = extract_file(paths_file)

# Read paths from paths file, convert to URLs
paths = read_every_line(paths_file_unzipped, 1e8)
print('{} lines extracted'.format(len(paths)))
path_urls = [url_prefix + path for path in paths]

302 lines extracted


In [11]:
# Save index files to their own folder
index_folder = os.path.join(storage_folder, 'index-files/')

if not os.path.isdir(index_folder):
    os.mkdir(index_folder)

In [12]:
index_folder

'../data/raw/index-files/'

In [14]:
# Visit each index URL, download & save chunks

n_chunks = 700 # round up from estimate of 669
i = 0

start_time = time.time()
for url in path_urls:
    # Derive save-to path from url
    save_file_name = url.split('/')[-1].replace('.gz', '')
    
    if 'cdx-' in save_file_name:
        save_path = os.path.join(index_folder, save_file_name)

        # Open a stream Request to the URL, define generator
        r = requests.get(url, stream=True)
        data_stream = decompress_stream(r.iter_content(1024))

        # Create file, wrtie data chunks to it:
        save_file = open(save_path, 'wb')

        print('Downloading {}'.format(save_file_name))

        for i in range(n_chunks):
            chunk = next(data_stream)
            if chunk:
                save_file.write(chunk)

        # Close file & Request connection
        save_file.close()
        r.close()
        print('Download complete!')
    i += 1

duration = (time.time() - start_time) / 60
print('Elapsed time: {:.0f} minutes'.format(duration))

Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download com

Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download complete!
Download com

error: Error -3 while decompressing data: incorrect header check

In [34]:
from pandas.io.json import json_normalize

import logging
logging.basicConfig(level=logging.DEBUG)

In [35]:
cols_to_keep = ['ts', 'url', 'languages']

def parse_index_file(file_name):
    # Read data from file
    with open(file_name, 'rb') as f:
        data = f.read()
    
    # Decode data
    string_data = data.decode('UTF-8')  
    
    # Parse data
    lines = string_data.split('\n')
    file_data = []
    
    for line in lines:
        try:
            ts = line.split('{')[0].split()[-1] # timestamp
            line_json = '{' + line.split('{')[-1]
            try:
                line_data = json.loads(line_json)
                line_data.update({'ts': ts})
            except:
                logging.debug('Error parsing json data {} in file {}'.format(line_json, file_name))
                line_data = {}
        except IndexError:
            logging.debug('Error parsing line {} in file {}'.format(line, file_name))
            line_data = {}
        
        file_data.append(line_data)
        
    df = json_normalize(file_data)
    df = df.dropna().drop_duplicates().reset_index(drop=True)
    df = df.drop(columns=[col for col in df.columns if col not in cols_to_keep])
    
    return df

In [36]:
i = 0
master_df = pd.DataFrame()

for file in os.listdir(index_folder)[:5]:
    print('Parsing index file {}...'.format(file))
    file_path = os.path.join(index_folder, file)
    df = parse_index_file(file_path)
    master_df = pd.concat([master_df, df])

print(len(master_df))

DEBUG:root:Error parsing line  in file ../data/raw/index-files/cdx-00111


Parsing index file cdx-00111...
Parsing index file cdx-00129...


DEBUG:root:Error parsing line  in file ../data/raw/index-files/cdx-00129
DEBUG:root:Error parsing line  in file ../data/raw/index-files/cdx-00116
DEBUG:root:Error parsing line  in file ../data/raw/index-files/cdx-00120


Parsing index file cdx-00116...
Parsing index file cdx-00120...


DEBUG:root:Error parsing line  in file ../data/raw/index-files/cdx-00118


Parsing index file cdx-00118...
0


In [58]:
cols_to_keep = ['ts', 'url', 'languages']

def parse_index_file(file_name):
    # Read data from file
    with open(file_name, 'rb') as f:
        data = f.read()
    
    # Decode data
    string_data = data.decode('UTF-8')  
    
    # Parse data
    lines = string_data.split('\n')
    file_data = []
    
    for line in lines:
        if line:
            ts = line.split('{')[0].split()[-1] # timestamp
            line_json = '{' + line.split('{')[-1]
            try:
                line_data = json.loads(line_json)
                line_data.update({'ts': ts})
            except:
                logging.debug('Error parsing json data {} in file {}'.format(line_json, file_name))
                line_data = {}

            file_data.append(line_data)
    
    # Covert from JSON to DataFrame
#     print('...Lines extracted: {}'.format(len(file_data)))
    df = json_normalize(file_data)
    df = df.drop_duplicates().reset_index(drop=True)
    df = df.drop(columns=[col for col in df.columns if col not in cols_to_keep])
    
    return df

In [53]:
import feather

In [73]:
i = 1
master_df = pd.DataFrame()

for file in os.listdir(index_folder):
    if 'cdx-' in file:
        if i % 10 == 0 or i == 1:
            sys.stdout.write('\rParsing file {} out of {}'.format(i, len(os.listdir(index_folder))))
            sys.stdout.flush()
        
        file_path = os.path.join(index_folder, file)
        df = parse_index_file(file_path)
        master_df = pd.concat([master_df, df], sort=False)
        i += 1
    
print('Parsing complete! {} total records extracted.'.format(len(master_df)))

logging.debug('Saving DataFrame...')
data_path = '../data/raw/'
master_file = 'cc_urls_' + yearmonth
feather.write_dataframe(master_df, os.path.join(data_path, master_file))

Parsing file 280 out of 301

DEBUG:root:Error parsing json data {"url": "https://signon.thomsonreuters.com/?productid=CBT&viewproductid=CBTOW&lr=0&culture=en-US&returnto=https%3A%2F%2Fcontent.next.westlaw.com%2FCosi%2FSignOn%3FredirectTo%3D%252fDocument%252fI6d4e31b0c20d11e19ef7851aa54f6629%252fView%252fFullText.html%253fcontextData%253d(sc.Default)%2526transitionType%253dDefault%2526firstPage%253dtrue&tracetoken=1122192045220BqigbyLpOpe1MqgzcW15lvqU844T5bR731cyNJLlWHotJZKqMko93dHIPy5IeqtRvIe2dseY7j09WZCZ5rE28BYy2WMtQu9mDu-r0b1Jp_J9wOgj8ckB7D-G-vpc7gi-M7mITDgTFvWWlegAB5kBfq2VDGTeTDDzASjnHaktgNljZkAll3LvsPfB59nme in file ../data/raw/index-files/cdx-00153


Parsing file 300 out of 301

DEBUG:root:Saving DataFrame...


Parsing complete! 898117 total records extracted.


This is not quite the 2.8 Million I'd hoped for

In [74]:
master_df.head()

,languages,ts,url
0,eng,20191114225518,https://myheatsinks.com/contact/rfq/?pn=S805-5...
1,NaN,20191113161053,http://www.myheatsinks.com/contact/rfq/?pn=S80...
2,eng,20191113161054,https://myheatsinks.com/contact/rfq/?pn=S805-6...
3,NaN,20191113161054,https://www.myheatsinks.com/contact/rfq/?pn=S8...
4,NaN,20191113161116,http://www.myheatsinks.com/contact/rfq/?pn=S80...
